In [1]:
import numpy as np
import AirSimClient
import time
import drone_env
import torch
import models
import cv2

import DDPG

In [2]:
           
np.set_printoptions(precision=3, suppress=True)


In [3]:
settings = {}
settings["learning_rate"] = 0.001
settings["reward_decay"] = 0.9
settings["buffer_length"] = 500000
settings["batch_size"] = 32
settings["model_name"] = None
settings["TAU"] = 0.001
settings["state_dim"] = [[1,56,56],[1]]
settings["action_dim"] = 1
settings["action_lim"] = 1

#actor = models.Actor(settings["state_dim"], settings["action_dim"], settings["action_lim"])
#critic = models.Critic(settings["state_dim"], settings["action_dim"])
actor,critic = DDPG.load("test")

settings["Actor"] = actor
settings["Critic"] = critic



In [4]:
def trans(state):
    res = np.array([])
    #for one in state:
    #    res = np.append(one.flatten(),res)
    res = np.append(res,state[0].flatten()/255)
    res = np.append(res,state[1].flatten())
    return res

def show():
    rawImage = env.client.simGetImage(0, AirSimClient.AirSimImageType.DepthVis)
    if (rawImage == None):
        pass
    else:
        png = cv2.imdecode(AirSimClient.AirSimClientBase.stringToUint8Array(rawImage), cv2.IMREAD_UNCHANGED)
        cv2.imshow("Depth", png)

In [5]:
RL = DDPG.DDPG(settings)
env = drone_env.drone_env_heightcontrol(view = True)


current = 0
count = 0
learns = 0
episode = 0
succ = 0
learn = 0
treward = []
rc = 0

Waiting for connection: 


In [6]:
s = trans(env.reset())
while True:
    
    action = RL.choose_action(s)
    #print (action)
    s_,r,done,info = env.step(action)
    #show()
    s_ = trans(s_)
    env.render(extra1 = str(count),extra2 = "reward: "+str(int(r))+" Done: "+str(succ)+" Episode: "+str(episode)+" Action:"+str(action)+" State:"+str(s_[-1]))
      
    count += 1
    current += 1
    if info == "success":
        succ += 1
    if current>100:
        done = True
        info = "out of steps"
    
    rc += r
    counter = RL.add_data(s, action, [r], s_)
    
    if done or current>200:
        episode += 1
        DDPG.save(RL.target_actor,RL.target_critic)
        treward.append(rc)
        print ("reset due to",info,"episode reward:",rc/current,"duration:",current)
        print ("--------------------------------reset---------------------------")
        current = 0
        rc = 0
        env.reset()
        
    if count > 1:
        RL.learn(times = 2)
        learn += 1
        
    s = s_

0 height: -8 difference: 1 reward: 78 Done: 0 Episode: 0 Action:[-0.512] State:1.09662914276
1 height: -7 difference: 2 reward: 49 Done: 0 Episode: 0 Action:[-0.625] State:2.54719686508
2 height: -5 difference: 4 reward: 21 Done: 0 Episode: 0 Action:[-0.592] State:3.92393493652
3 height: -4 difference: 5 reward: -2 Done: 0 Episode: 0 Action:[-0.49] State:5.10007810593
4 height: -3 difference: 6 reward: -22 Done: 0 Episode: 0 Action:[-0.42] State:6.12077140808
5 height: -2 difference: 7 reward: -46 Done: 0 Episode: 0 Action:[-0.538] State:7.33463668823
6 height: -1 difference: 8 reward: -100 Done: 0 Episode: 0 Action:[-0.485] State:8.1019821167
models saved
reset due to collision episode reward: -3.20927756173 duration: 7
--------------------------------reset---------------------------
7 height: -8 difference: 1 reward: 68 Done: 0 Episode: 1 Action:[-0.844] State:1.55077552795
8 height: -6 difference: 3 reward: 38 Done: 0 Episode: 1 Action:[-0.626] State:3.05814123154
9 height: -5 diffe

73 height: -10 difference: 0 reward: 85 Done: 0 Episode: 9 Action:[ 0.645] State:-0.705181121826
74 height: -12 difference: -2 reward: 53 Done: 0 Episode: 9 Action:[ 0.545] State:-2.34842681885
75 height: -13 difference: -3 reward: 27 Done: 0 Episode: 9 Action:[ 0.438] State:-3.63755226135
76 height: -15 difference: -5 reward: -5 Done: 0 Episode: 9 Action:[ 0.695] State:-5.29055976868
77 height: -17 difference: -7 reward: -41 Done: 0 Episode: 9 Action:[ 0.617] State:-7.07028579712
78 height: -18 difference: -8 reward: -66 Done: 0 Episode: 9 Action:[ 0.37] State:-8.33942604065
79 height: -19 difference: -9 reward: -79 Done: 0 Episode: 9 Action:[ 0.189] State:-8.9912147522
80 height: -19 difference: -9 reward: -91 Done: 0 Episode: 9 Action:[ 0.264] State:-9.56460952759
81 height: -20 difference: -10 reward: -100 Done: 0 Episode: 9 Action:[ 0.171] State:-10.0067024231
82 height: -20 difference: -10 reward: -100 Done: 0 Episode: 9 Action:[ 0.391] State:-10.791929245
models saved
reset due 

142 height: -18 difference: -8 reward: -78 Done: 0 Episode: 19 Action:[ 0.592] State:-8.94072532654
143 height: -20 difference: -10 reward: -109 Done: 0 Episode: 19 Action:[ 0.572] State:-10.4879951477
144 height: -21 difference: -11 reward: -100 Done: 0 Episode: 19 Action:[ 0.45] State:-11.794303894
models saved
reset due to too high episode reward: -37.7791372935 duration: 6
--------------------------------reset---------------------------
145 height: -10 difference: 0 reward: 87 Done: 0 Episode: 20 Action:[ 0.594] State:-0.642972946167
146 height: -12 difference: -2 reward: 50 Done: 0 Episode: 20 Action:[ 0.663] State:-2.45803642273
147 height: -14 difference: -4 reward: 4 Done: 0 Episode: 20 Action:[ 0.917] State:-4.76850891113
148 height: -17 difference: -7 reward: -49 Done: 0 Episode: 20 Action:[ 1.154] State:-7.49808120728
149 height: -20 difference: -10 reward: -106 Done: 0 Episode: 20 Action:[ 1.005] State:-10.3302192688
150 height: -23 difference: -13 reward: -100 Done: 0 Epis

208 height: -11 difference: -1 reward: 79 Done: 0 Episode: 31 Action:[ 0.94] State:-1.04668712616
209 height: -13 difference: -3 reward: 33 Done: 0 Episode: 31 Action:[ 0.665] State:-3.34011936188
210 height: -15 difference: -5 reward: -10 Done: 0 Episode: 31 Action:[ 0.872] State:-5.54709243774
211 height: -18 difference: -8 reward: -64 Done: 0 Episode: 31 Action:[ 1.141] State:-8.21337890625
212 height: -21 difference: -11 reward: -122 Done: 0 Episode: 31 Action:[ 1.123] State:-11.1356048584
213 height: -23 difference: -13 reward: -100 Done: 0 Episode: 31 Action:[ 1.037] State:-13.9170684814
models saved
reset due to too high episode reward: -30.9429423014 duration: 6
--------------------------------reset---------------------------
214 height: -11 difference: -1 reward: 77 Done: 0 Episode: 32 Action:[ 1.31] State:-1.11480617523
215 height: -14 difference: -4 reward: 20 Done: 0 Episode: 32 Action:[ 1.522] State:-3.9587688446
216 height: -17 difference: -7 reward: -36 Done: 0 Episode: 

276 height: -11 difference: -1 reward: 77 Done: 0 Episode: 41 Action:[ 1.156] State:-1.11396789551
277 height: -14 difference: -4 reward: 20 Done: 0 Episode: 41 Action:[ 1.411] State:-3.9748249054
278 height: -16 difference: -6 reward: -35 Done: 0 Episode: 41 Action:[ 1.102] State:-6.77915382385
279 height: -19 difference: -9 reward: -91 Done: 0 Episode: 41 Action:[ 1.261] State:-9.57187843323
280 height: -22 difference: -12 reward: -100 Done: 0 Episode: 41 Action:[ 1.531] State:-12.550409317
models saved
reset due to too high episode reward: -25.7593002319 duration: 5
--------------------------------reset---------------------------
281 height: -11 difference: -1 reward: 77 Done: 0 Episode: 42 Action:[ 1.351] State:-1.11142158508
282 height: -14 difference: -4 reward: 20 Done: 0 Episode: 42 Action:[ 1.103] State:-3.97227478027
283 height: -16 difference: -6 reward: -25 Done: 0 Episode: 42 Action:[ 0.725] State:-6.26939582825
284 height: -18 difference: -8 reward: -59 Done: 0 Episode: 4

...

12341 height: -10 difference: 0 reward: 85 Done: 0 Episode: 2091 Action:[ 0.654] State:-0.713063240051
12342 height: -12 difference: -2 reward: 58 Done: 0 Episode: 2091 Action:[ 0.425] State:-2.08649253845
12343 height: -13 difference: -3 reward: 35 Done: 0 Episode: 2091 Action:[ 0.439] State:-3.23397636414
12344 height: -14 difference: -4 reward: 11 Done: 0 Episode: 2091 Action:[ 0.458] State:-4.40605258942
12345 height: -16 difference: -6 reward: -23 Done: 0 Episode: 2091 Action:[ 0.739] State:-6.1679058075
12346 height: -18 difference: -8 reward: -62 Done: 0 Episode: 2091 Action:[ 0.701] State:-8.12219810486
12347 height: -20 difference: -10 reward: -102 Done: 0 Episode: 2091 Action:[ 0.775] State:-10.1299209595
12348 height: -22 difference: -12 reward: -100 Done: 0 Episode: 2091 Action:[ 0.632] State:-11.9690513611
models saved
reset due to too high episode reward: -12.1490240097 duration: 8
--------------------------------reset---------------------------
12349 height: -10 differen

12407 height: -17 difference: -7 reward: -50 Done: 0 Episode: 2101 Action:[ 1.112] State:-7.51205635071
12408 height: -21 difference: -11 reward: -118 Done: 0 Episode: 2101 Action:[ 0.959] State:-10.9294013977
12409 height: -24 difference: -14 reward: -100 Done: 0 Episode: 2101 Action:[ 1.19] State:-14.4389381409
models saved
reset due to too high episode reward: -34.1460609436 duration: 5
--------------------------------reset---------------------------
12410 height: -11 difference: -1 reward: 77 Done: 0 Episode: 2102 Action:[ 1.034] State:-1.11345672607
12411 height: -14 difference: -4 reward: 21 Done: 0 Episode: 2102 Action:[ 1.058] State:-3.92926216125
12412 height: -16 difference: -6 reward: -35 Done: 0 Episode: 2102 Action:[ 1.018] State:-6.76175117493
12413 height: -19 difference: -9 reward: -90 Done: 0 Episode: 2102 Action:[ 1.242] State:-9.53195381165
12414 height: -22 difference: -12 reward: -100 Done: 0 Episode: 2102 Action:[ 1.13] State:-12.3359394073
models saved
reset due 

12471 height: -11 difference: -1 reward: 78 Done: 0 Episode: 2112 Action:[ 0.952] State:-1.0613861084
12472 height: -14 difference: -4 reward: 21 Done: 0 Episode: 2112 Action:[ 1.319] State:-3.92601394653
12473 height: -16 difference: -6 reward: -35 Done: 0 Episode: 2112 Action:[ 1.295] State:-6.76976394653
12474 height: -19 difference: -9 reward: -94 Done: 0 Episode: 2112 Action:[ 1.532] State:-9.7201423645
12475 height: -22 difference: -12 reward: -100 Done: 0 Episode: 2112 Action:[ 1.415] State:-12.552280426
models saved
reset due to too high episode reward: -25.9092254639 duration: 5
--------------------------------reset---------------------------
12476 height: -11 difference: -1 reward: 77 Done: 0 Episode: 2113 Action:[ 1.334] State:-1.1156167984
12477 height: -14 difference: -4 reward: 20 Done: 0 Episode: 2113 Action:[ 1.163] State:-3.97647380829
12478 height: -16 difference: -6 reward: -34 Done: 0 Episode: 2113 Action:[ 1.304] State:-6.73575782776
12479 height: -19 difference: -

12534 height: -15 difference: -5 reward: -4 Done: 0 Episode: 2124 Action:[ 0.586] State:-5.20738697052
12535 height: -16 difference: -6 reward: -36 Done: 0 Episode: 2124 Action:[ 0.622] State:-6.84704017639
12536 height: -19 difference: -9 reward: -77 Done: 0 Episode: 2124 Action:[ 0.804] State:-8.8995552063
12537 height: -21 difference: -11 reward: -121 Done: 0 Episode: 2124 Action:[ 0.742] State:-11.082244873
12538 height: -23 difference: -13 reward: -100 Done: 0 Episode: 2124 Action:[ 0.877] State:-13.287612915
models saved
reset due to too high episode reward: -31.5373883929 duration: 7
--------------------------------reset---------------------------
12539 height: -10 difference: 0 reward: 84 Done: 0 Episode: 2125 Action:[ 0.718] State:-0.794080734253
12540 height: -12 difference: -2 reward: 42 Done: 0 Episode: 2125 Action:[ 0.76] State:-2.85893058777
12541 height: -15 difference: -5 reward: -8 Done: 0 Episode: 2125 Action:[ 1.098] State:-5.41909599304
12542 height: -19 difference:

KeyboardInterrupt: 

In [ ]:
s1 = [0,1]
s2 = [0,-10]

In [ ]:
s[-1] = 0

In [12]:
RL.critic.forward(torch.autograd.Variable(torch.from_numpy(np.array([s])).float()),torch.autograd.Variable(torch.from_numpy(np.array([[0]])).float()))

Variable containing:
-235.0042
[torch.FloatTensor of size 1x1]

In [ ]:
s